In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import undetected_chromedriver as uc
from datetime import datetime
from time import sleep
import os



## Tạo hàm chuyển đổi Cookies

In [8]:
def convert_cookie_string_to_dict(cookie_string):
    cookie_dict = {}
    items = cookie_string.split(';')
    for item in items:
        if '=' in item:  # Kiểm tra xem có dấu '=' trong item không
            key, value = item.strip().split('=', 1)
            cookie_dict[key] = value
    return cookie_dict

a = "REC_T_ID=00242cbd-931c-11ee-be6d-ba8ff0e0943f;SPC_F=3X0jeS85aHy10x5rodCSg42OXHfLudq9;_fbp=fb.1.1701745895419.1410008333;_ga_M32T05RVZT=GS1.1.1701847999.2.1.1701849865.60.0.0;_hjSessionUser_868286=eyJpZCI6IjE2MzcyZjc5LTNkNDYtNTkwZC1iNjI0LTEwMTMwNTJlMTUzNyIsImNyZWF0ZWQiOjE3MDE3NDYxMDU4NTQsImV4aXN0aW5nIjp0cnVlfQ==;SPC_CLIENTID=M1gwamVTODVhSHkxetweujfvqupjahmc;SC_DFP=TaIeyIvQNeXtGVmLjdYrKOsPNApiradQ;_ga_3XVGTY3603=GS1.1.1709352133.3.1.1709352391.60.0.0;_gcl_au=1.1.673493001.1709522456;_med=affiliates;SPC_SI=zBfwZQAAAAB6TkllU3Q4WiSfvgAAAAAAWDNEQkthUVk=;SPC_SEC_SI=v1-MnFzb2FIYTJIVWhuYUs1bNg+piZ0ANCas2tXgQcIcADcaU13flh85tcHvteXV4lV8CZfhJ7ADcfqO4629fwXc710xKmwoNMAIUJmuf8wfE8=;_QPWSDCXHZQA=6efefc3e-78da-4e0b-e40d-8aeb6a9f2d17;REC7iLP4Q=966f9dcc-dbef-4b80-9572-a74b08014da0;_gid=GA1.2.1865841269.1710854682;SPC_U=142941210;SPC_T_ID=tA5/FdAhAGKDIWNR4lg9QIsSqt2t5MA0mbx51YJggCCJe9ZbQjq/KxJgjDkISqf2/1vcOI1C95J4tmuoAlhAZK/0feCM10vkif80VfO3dU0ueB38e1+QKXwPd6ay5+wWplV5z9KaxaP5LdnJLX8FbfQr2P62D3bLaFW7teUXMms=;SPC_T_IV=Uzh2SzhDOXNvNHZHSFl4dg==;SPC_R_T_ID=tA5/FdAhAGKDIWNR4lg9QIsSqt2t5MA0mbx51YJggCCJe9ZbQjq/KxJgjDkISqf2/1vcOI1C95J4tmuoAlhAZK/0feCM10vkif80VfO3dU0ueB38e1+QKXwPd6ay5+wWplV5z9KaxaP5LdnJLX8FbfQr2P62D3bLaFW7teUXMms=;SPC_R_T_IV=Uzh2SzhDOXNvNHZHSFl4dg==;__LOCALE__null=VN;csrftoken=70obbTyMPs7Y5EKOefrxBjfCyVK5kl4a;_sapid=9509f6ffb058729a5c3884ce9ced2f8f1605b70813d4c7dd341f156c;_hjSession_868286=eyJpZCI6IjYwOWZhNzA1LWU1NzMtNDEwNS04ZmVjLTZjODNjMzY5MDk1YyIsImMiOjE3MTA4NjgwOTcwMTgsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=;AMP_TOKEN=%24NOT_FOUND;_dc_gtm_UA-61914164-6=1;SPC_EC=.RVM3ZWZnc0paM0xKYm9LcXgZfOILvQ3x6IPHR0EktgFBiHWG0kDD1W3e0vocxme24VSqBDjoAdAUyOTxJjP39eKUohwn7wiLxEpx3MNakQUXt8b1+ze5DKVlF9DXAouOftCEVinueG2kOy4dRpEY/D65UkLYO3EcWgJHanh+UxayAwNAp+CXEyrMytREf0ZiTyMby3BN9vqn21kYHNcXLQ==;SPC_ST=.RVM3ZWZnc0paM0xKYm9LcXgZfOILvQ3x6IPHR0EktgFBiHWG0kDD1W3e0vocxme24VSqBDjoAdAUyOTxJjP39eKUohwn7wiLxEpx3MNakQUXt8b1+ze5DKVlF9DXAouOftCEVinueG2kOy4dRpEY/D65UkLYO3EcWgJHanh+UxayAwNAp+CXEyrMytREf0ZiTyMby3BN9vqn21kYHNcXLQ==;_ga_4GPP1ZXG63=GS1.1.1710868097.62.1.1710868108.49.0.0;_ga=GA1.1.1033809469.1701745897;shopee_webUnique_ccd=y9HaqnsmQqbCPXNPcDZ0Lg%3D%3D%7Ca3PdkCMbbbl%2FDz%2FcWNHgdPgeE%2B9r51IViQCfTLsz%2BKFQPG11VoxW6dSIyW5GKd6XWjVFRfYF7zXYr9w%3D%7CRvVDJv1TYWT3MXQZ%7C08%7C3;ds=60774d50c14ab73536f7e72570c63291;|Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
# Sử dụng hàm đã định nghĩa để chuyển đổi
cookies = convert_cookie_string_to_dict(a)
cookies

{'REC_T_ID': '00242cbd-931c-11ee-be6d-ba8ff0e0943f',
 'SPC_F': '3X0jeS85aHy10x5rodCSg42OXHfLudq9',
 '_fbp': 'fb.1.1701745895419.1410008333',
 '_ga_M32T05RVZT': 'GS1.1.1701847999.2.1.1701849865.60.0.0',
 '_hjSessionUser_868286': 'eyJpZCI6IjE2MzcyZjc5LTNkNDYtNTkwZC1iNjI0LTEwMTMwNTJlMTUzNyIsImNyZWF0ZWQiOjE3MDE3NDYxMDU4NTQsImV4aXN0aW5nIjp0cnVlfQ==',
 'SPC_CLIENTID': 'M1gwamVTODVhSHkxetweujfvqupjahmc',
 'SC_DFP': 'TaIeyIvQNeXtGVmLjdYrKOsPNApiradQ',
 '_ga_3XVGTY3603': 'GS1.1.1709352133.3.1.1709352391.60.0.0',
 '_gcl_au': '1.1.673493001.1709522456',
 '_med': 'affiliates',
 'SPC_SI': 'zBfwZQAAAAB6TkllU3Q4WiSfvgAAAAAAWDNEQkthUVk=',
 'SPC_SEC_SI': 'v1-MnFzb2FIYTJIVWhuYUs1bNg+piZ0ANCas2tXgQcIcADcaU13flh85tcHvteXV4lV8CZfhJ7ADcfqO4629fwXc710xKmwoNMAIUJmuf8wfE8=',
 '_QPWSDCXHZQA': '6efefc3e-78da-4e0b-e40d-8aeb6a9f2d17',
 'REC7iLP4Q': '966f9dcc-dbef-4b80-9572-a74b08014da0',
 '_gid': 'GA1.2.1865841269.1710854682',
 'SPC_U': '142941210',
 'SPC_T_ID': 'tA5/FdAhAGKDIWNR4lg9QIsSqt2t5MA0mbx51YJggCCJe9ZbQjq

### Iphone: https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-cat.11036030.11036031?brands=1189223&page={j}&sortBy=pop
### SamSung: https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-cat.11036030.11036031?brands=1695294&page={j}&sortBy=pop

## Links

In [10]:
def add_cookies(driver, cookies):
    driver.get("https://shopee.vn")  # Mở Shopee để thêm cookies
    sleep(3)  # Đợi trang tải xong
    for name, value in cookies.items():
        driver.add_cookie({'name': name, 'value': value})
    driver.refresh()  # Tải lại trang sau khi thêm cookies
    print("Cookies added successfully.")

def safe_find_element(driver, selector, by=By.CSS_SELECTOR):
    try:
        element = driver.find_element(by, selector)
        return element
    except NoSuchElementException:
        return None

def get_product_links(driver, j):
    product_links = []  # Danh sách để lưu trữ các link sản phẩm
    url = f'https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-cat.11036030.11036031?brands=1695294&page={j}&sortBy=pop'
    driver.get(url)
    sleep(4.765)  # Adjust based on your actual wait time needed for the page to load

    try:
        for i in range(1, 61):  # Thay đổi số lượng lặp nếu cần
            try:
                # Cập nhật selector để phản ánh đúng cấu trúc của trang web
                selector = f"#main > div > div:nth-child(3) > div > div > div.container.cZOmU2 > div.uXYBTq > section > ul > li:nth-child({i}) > a"
                element = driver.find_element(By.CSS_SELECTOR, selector)
                product_link = element.get_attribute('href')
                if product_link:
                    product_links.append(product_link)
                    print(product_link)
            except NoSuchElementException:
                print(f"Phần tử thứ {i} không được tìm thấy.")
                continue  # Dừng vòng lặp nếu phần tử không được tìm thấy

        return product_links
    except TimeoutException:
        print("Timed out waiting for page to load")
        return []


In [11]:
# Khởi tạo undetected_chromedriver với các tuỳ chỉnh
options = uc.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
driver.implicitly_wait(10)

# Thêm cookies vào trình duyệt
add_cookies(driver, cookies)

product_links = []
for j in range(0,17):  # Giả sử bạn chỉ muốn lấy dữ liệu từ trang đầu tiên
    product_links.extend(get_product_links(driver, j))

driver.quit()

# Xuất dữ liệu ra file CSV
products_df = pd.DataFrame(product_links, columns=['URL'])
print(products_df)

Cookies added successfully.
https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Samsung-Galaxy-A14-5G-(4GB-128GB)-H%C3%A0ng-ch%C3%ADnh-h%C3%A3ng-i.85762618.23981430230?sp_atk=9d3b36f1-7008-4c76-ab5b-1086fd6733f5&xptdk=9d3b36f1-7008-4c76-ab5b-1086fd6733f5
https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%A1i-Samsung-Galaxy-S23-Ultra-5G-512GB-H%C3%A0ng-Ch%C3%ADnh-H%C3%A3ng-i.1037677547.23977882901?sp_atk=a4bf7817-eef1-475a-8fcc-e3ebd42d23d6&xptdk=a4bf7817-eef1-475a-8fcc-e3ebd42d23d6
https://shopee.vn/Camera-SS-A750-A7-2018-h%C3%A0ng-b%E1%BB%91c-m%C3%A1y-zin-%C4%91%E1%BA%B9p-%C4%91%E1%BB%A7-b%E1%BB%99-i.942951237.23977654410?sp_atk=571f8aed-4432-414e-8742-eda239e5ccca&xptdk=571f8aed-4432-414e-8742-eda239e5ccca
https://shopee.vn/%C4%91i%E1%BB%87n-tho%E1%BA%A1i-samsung-s21-plus-m%C3%A1y-%C4%91%E1%BA%B9p-nguy%C3%AAn-zin-b%E1%BA%A3n-m%E1%BB%B9-2-sim-ram-8-128g-chip-snackdragon-888-i.580308992.23976560709?sp_atk=dd593ee9-0212-43ca-b112-01668892c69d&xptdk=dd593ee9-0212-43ca-b112-01668892c69d
https://

In [12]:
# Xử lý lọc Iphone15
def filter_apple_iphone_15_links(products_df):
    filtered_df = products_df[products_df['URL'].str.contains("Apple-iPhone-15", case=False, na=False)]
    return filtered_df

# Xử lý lọc SamSung S23
def filter_samsung_galaxy_s23_links(products_df):
    # Sử dụng .str.contains() để lọc các hàng có đường link chứa từ "Samsung-Galaxy-S23"
    filtered_df = products_df[products_df['URL'].str.contains("Samsung-Galaxy-S23", case=False, na=False)]
    return filtered_df

In [8]:
filtered_products_iphone_df = filter_apple_iphone_15_links(products_df)
print(filtered_products_iphone_df)

                                                  URL
16  https://shopee.vn/Apple-iPhone-15-Plus-128GB-i...
17  https://shopee.vn/Apple-iPhone-15-Pro-Max-256G...
21  https://shopee.vn/-Tr%E1%BA%A3-g%C3%B3p-0-Appl...
23  https://shopee.vn/Apple-iPhone-15-Pro-Max-512G...
24  https://shopee.vn/Apple-iPhone-15-Pro-128GB-Ch...
25  https://shopee.vn/Apple-iPhone-15-Pro-512GB-Ch...
26  https://shopee.vn/Apple-iPhone-15-128GB-i.2882...
28  https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
30  https://shopee.vn/Apple-iPhone-15-128GB-Ch%C3%...
34  https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
37  https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
38  https://shopee.vn/-M%C3%A3-ELBSMG03-gi%E1%BA%A...
39  https://shopee.vn/Apple-iPhone-15-Pro-128GB-i....
45  https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
48  https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
60  https://shopee.vn/Apple-iPhone-15-Pro-256GB-Ch...


In [13]:
filtered_products_Samsung_df = filter_samsung_galaxy_s23_links(products_df)
print(filtered_products_Samsung_df)

                                                    URL
1     https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
11    https://shopee.vn/(b%E1%BA%A3o-h%C3%A0nh-1-%C4...
20    https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
22    https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
55    https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
74    https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
88    https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
109   https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
128   https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
134   https://shopee.vn/-Ch%C3%ADnh-H%C3%A3ng-%C4%90...
140   https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
173   https://shopee.vn/(b%E1%BA%A3o-h%C3%A0nh-1-%C4...
190   https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
192   https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
353   https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
354   https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
361   https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1

In [14]:
filtered_products_Samsung_df.to_csv('links_smartphone_shopee_19-3.csv', mode='a', header=not os.path.exists('links_smartphone_shopee_19-3.csv'), index=False, encoding='utf-8-sig')

In [9]:
filtered_products_iphone_df.to_csv('links_smartphone_shopee_19-3.csv', mode='a', header=not os.path.exists('links_smartphone_shopee_19-3.csv'), index=False, encoding='utf-8-sig')

## Crawl thông tin sản phẩm

In [24]:
def add_cookies(driver, cookies):
    driver.get("https://shopee.vn")  # Mở Shopee để thêm cookies
    sleep(3)  # Đợi trang tải xong
    for name, value in cookies.items():
        driver.add_cookie({'name': name, 'value': value})
    driver.refresh()  # Tải lại trang sau khi thêm cookies
    print("Cookies added successfully.")

def get_comment(driver, link):
    products = [] # Danh sách để lưu trữ các comment
    driver.get(link)
    sleep(5)  # Đợi trang tải và cookies có hiệu lực

    print(link)

    try:
        # Lấy ngày hiện tại
        current_date = datetime.now().strftime("%Y-%m-%d")
        print(current_date)
        
        product_title = safe_find_text(driver, '#main > div > div:nth-child(3) > div:nth-child(1) > div.KrtGbA > div > div > section.product-briefing.flex.card.vX9SYw > section.flex.flex-auto.i9t0tr > div > div.WBVL_7 > span')
        print("Product Title:", product_title)

        brand = safe_find_text(driver, '#main > div > div:nth-child(3) > div.theme--ofs > div.KrtGbA > div > div > div.wAMdpk > div > div.page-product__content--left > div.product-detail.page-product__detail > section:nth-child(1) > div > div:nth-child(2) > a')
        print("Brand:", brand)          #main > div > div:nth-child(3) > div.theme--ofs > div.KrtGbA > div > div > div.wAMdpk > div > div.page-product__content--left > div.product-detail.page-product__detail > section:nth-child(1) > div > div:nth-child(2) > a
                                        #main > div > div:nth-child(3) > div.theme--ofs > div.KrtGbA > div > div > div.wAMdpk > div > div.page-product__content--left > div.product-detail.page-product__detail > section:nth-child(1) > div > div:nth-child(2) > a
        current_price = safe_find_text(driver, '#main > div > div:nth-child(3) > div:nth-child(1) > div.KrtGbA > div > div > section.product-briefing.flex.card.vX9SYw > section.flex.flex-auto.i9t0tr > div > div:nth-child(3) > div > div > section > div > div.qg2n76')
        
        discounted_price = safe_find_text(driver, '#main > div > div:nth-child(3) > div:nth-child(1) > div.KrtGbA > div > div > section.product-briefing.flex.card.vX9SYw > section.flex.flex-auto.i9t0tr > div > div:nth-child(3) > div > div > section > div > div.flex.items-center > div.G27FPf')
        
        discount = safe_find_text(driver, '#main > div > div:nth-child(3) > div:nth-child(1) > div.KrtGbA > div > div > section.product-briefing.flex.card.vX9SYw > section.flex.flex-auto.i9t0tr > div > div:nth-child(3) > div > div > section > div > div.flex.items-center > div.o_z7q9')
        print("discount:", discount)
        
        colors = []
        try:
            for i in range(1, 6):  # Lặp từ 1 đến 5
                # Tạo XPath động với giá trị i
                xpath = f'/html/body/div[1]/div/div[2]/div[1]/div[1]/div/div/section[1]/section[2]/div/div[4]/div/div/div/section[1]/div/button[{i}]'
                # Sử dụng XPath để tìm phần tử
                element = driver.find_element(By.XPATH, xpath)
                # Lấy giá trị của thuộc tính aria-label
                aria_label_value = element.get_attribute('aria-label')
                
                # Thêm giá trị vào list colors
                if aria_label_value:  # Kiểm tra xem giá trị có tồn tại không trước khi thêm
                    colors.append(aria_label_value)
        except NoSuchElementException:
            print("Phần tử không được tìm thấy, thoát khỏi vòng lặp.")
        print(colors)


        sold_quanlity = safe_find_text(driver, '/html/body/div[1]/div/div[2]/div[1]/div[1]/div/div/section[1]/section[2]/div/div[2]/div/div[1]', By.XPATH)
        print(sold_quanlity)

        rating_averrage = safe_find_text(driver, '/html/body/div[1]/div/div[2]/div[1]/div[1]/div/div/section[1]/section[2]/div/div[2]/button[1]/div[1]', By.XPATH)
        print(rating_averrage)

        rating_count = safe_find_text(driver, '/html/body/div[1]/div/div[2]/div[1]/div[1]/div/div/section[1]/section[2]/div/div[2]/button[2]/div[1]', By.XPATH)
        print(rating_count)

        store = safe_find_text(driver, '#main > div > div:nth-child(3) > div:nth-child(1) > div.KrtGbA > div > div > section.r74CsV.page-product__shop > div.uLQaPg > div > div.fV3TIn')
        print(store)

        product_info = {
            'Platform': 'Shopee',
            'URL': link,
            'Date Crawled': current_date,
            'Store': store,
            'Product Title': product_title,
            'brand': brand,
            'Current Price': current_price,
            'Discounted Price': discounted_price,
            'Discount' : discount,
            'Available Colors': colors,
            'sold_quanlity': sold_quanlity,
            'rating_averrage': rating_averrage,
            'rating_count': rating_count,
        }
        
        products.append(product_info)
    except TimeoutException:
        print("Timed out waiting for page to load")
    finally:
        return products

def safe_find_text(driver, selector, by=By.CSS_SELECTOR):
    try:
        element = driver.find_element(by, selector)
        return element.text
    except NoSuchElementException:
        return None


In [16]:
link = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang TMĐT\Shopee\links_smartphone_shopee_19-3.csv")

In [19]:
links = link['URL'].to_list()
links

['https://shopee.vn/Apple-iPhone-15-Plus-128GB-i.288286284.23555130713?sp_atk=d60a40a1-53ca-40c1-a3f3-38bbc882deee&xptdk=d60a40a1-53ca-40c1-a3f3-38bbc882deee',
 'https://shopee.vn/Apple-iPhone-15-Pro-Max-256GB-Ch%C3%ADnh-h%C3%A3ng-VN-A-i.308461157.23354704062?sp_atk=98f5b9c0-3fb9-4706-b764-4778583d8f2f&xptdk=98f5b9c0-3fb9-4706-b764-4778583d8f2f',
 'https://shopee.vn/-Tr%E1%BA%A3-g%C3%B3p-0-Apple-iPhone-15Plus-256GB-H%C3%A0ng-Ch%C3%ADnh-H%C3%A3ng-Futureworld-APR--i.92532150.22983256043?sp_atk=5afd03ff-79e8-400b-832f-2cdbfd2ab07d&xptdk=5afd03ff-79e8-400b-832f-2cdbfd2ab07d',
 'https://shopee.vn/Apple-iPhone-15-Pro-Max-512GB-Ch%C3%ADnh-h%C3%A3ng-VN-A-i.308461157.22580204951?sp_atk=515ae391-7db4-42d3-bfed-a03d8aea5b88&xptdk=515ae391-7db4-42d3-bfed-a03d8aea5b88',
 'https://shopee.vn/Apple-iPhone-15-Pro-128GB-Ch%C3%ADnh-h%C3%A3ng-VN-A-i.308461157.22180197277?sp_atk=256de47f-d920-445b-b76a-700a3ede13d5&xptdk=256de47f-d920-445b-b76a-700a3ede13d5',
 'https://shopee.vn/Apple-iPhone-15-Pro-512GB-C

In [25]:
# Khởi tạo undetected_chromedriver với các tuỳ chỉnh
options = uc.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
driver.implicitly_wait(10)

# Thêm cookies vào trình duyệt
add_cookies(driver, cookies)

products = []
for link in links:
    products.extend(get_comment(driver, link))

# Sau khi đã lấy xong dữ liệu, đóng trình duyệt
driver.quit()

# Tạo DataFrame từ danh sách comments và xuất ra CSV
product_df = pd.DataFrame(products)

print(product_df)

Cookies added successfully.
https://shopee.vn/Apple-iPhone-15-Plus-128GB-i.288286284.23555130713?sp_atk=d60a40a1-53ca-40c1-a3f3-38bbc882deee&xptdk=d60a40a1-53ca-40c1-a3f3-38bbc882deee
2024-03-19
Product Title: Apple iPhone 15 Plus 128GB
Brand: Apple
discount: 21% GIẢM
['Blue', 'Black', 'Pink', 'Green', 'Yellow']
354
4.9
77
ShopDunk Official Store
https://shopee.vn/Apple-iPhone-15-Pro-Max-256GB-Ch%C3%ADnh-h%C3%A3ng-VN-A-i.308461157.23354704062?sp_atk=98f5b9c0-3fb9-4706-b764-4778583d8f2f&xptdk=98f5b9c0-3fb9-4706-b764-4778583d8f2f
2024-03-19
Product Title: Apple iPhone 15 Pro Max 256GB Chính hãng VN/A
Brand: Apple
discount: 15% GIẢM
Phần tử không được tìm thấy, thoát khỏi vòng lặp.
['Natural Titanium', 'Blue Titanium', 'White Titanium', 'Black Titanium']
5,9k
5.0
1,1k
Viettel Store - AAR
https://shopee.vn/-Tr%E1%BA%A3-g%C3%B3p-0-Apple-iPhone-15Plus-256GB-H%C3%A0ng-Ch%C3%ADnh-H%C3%A3ng-Futureworld-APR--i.92532150.22983256043?sp_atk=5afd03ff-79e8-400b-832f-2cdbfd2ab07d&xptdk=5afd03ff-79e8-4

In [3]:
def convert_to_number(x):
    if isinstance(x, str):  # Kiểm tra nếu x là chuỗi
        x = x.replace(',', '').lower()  # Loại bỏ dấu phẩy và chuyển về chữ thường
        if 'k' in x:
            x = x.replace('k', '')  # Loại bỏ chữ 'k'
            return float(x) * 100  # Nhân với 100
        else:
            return float(x)  # Chuyển chuỗi thành số
    return x

# Áp dụng hàm chuyển đổi cho cột rating_count
product_df['rating_count'] = product_df['rating_count'].apply(convert_to_number)

def filter_rating_count(df, threshold=50):
    filtered_df = df[df['rating_count'] > threshold]
    return filtered_df
# Sử dụng hàm để lọc dữ liệu
filtered_data = filter_rating_count(product_df)

# Hiển thị dữ liệu sau khi lọc
print(filtered_data)

   Platform                                                URL Date Crawled  \
0    Shopee  https://shopee.vn/Apple-iPhone-15-Plus-128GB-i...    3/19/2024   
1    Shopee  https://shopee.vn/Apple-iPhone-15-Pro-Max-256G...    3/19/2024   
4    Shopee  https://shopee.vn/Apple-iPhone-15-Pro-128GB-Ch...    3/19/2024   
7    Shopee  https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...    3/19/2024   
9    Shopee  https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...    3/19/2024   
10   Shopee  https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...    3/19/2024   
12   Shopee  https://shopee.vn/Apple-iPhone-15-Pro-128GB-i....    3/19/2024   
13   Shopee  https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...    3/19/2024   
14   Shopee  https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...    3/19/2024   
15   Shopee  https://shopee.vn/Apple-iPhone-15-Pro-256GB-Ch...    3/19/2024   
32   Shopee  https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...    3/19/2024   
45   Shopee  https://shopee.vn/%C4%90i%E1%BB%87n-Tho

In [4]:
filtered_data.to_csv('product_smartphone_shopee_19-3.csv', mode='a', header=not os.path.exists('product_smartphone_shopee_19-3.csv'), index=False, encoding='utf-8-sig')

# Get attributes from comments

In [10]:
# Định nghĩa các XPATHs
xpaths = [
    "/html/body/div[1]/div/div[2]/div[1]/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div/div[2]/div[2]/div[7]",
    "/html/body/div[1]/div/div[2]/div[1]/div[1]/div/div/div[3]/div/div[1]/div[2]/div/div/div[2]/div[2]/div[7]"
]
def add_cookies(driver, cookies):
    driver.get("https://shopee.vn")  # Mở Shopee để thêm cookies
    for name, value in cookies.items():
        driver.add_cookie({'name': name, 'value': value})
    print("Cookies added successfully.")

def get_comment(driver, link):
    comments_list = []  # Danh sách để lưu trữ các comment
    driver.get(link)
    sleep(10)  # Đợi trang tải và cookies có hiệu lực

    print(link)
    # Điều hướng đến Phần Tổng Quan Bình Luận
    try:
        # Đợi cho đến khi phần tổng quan bình luận được tải
        comment_overview = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.product-rating-overview')))
        # Scroll đến phần tổng quan bình luận
        driver.execute_script("arguments[0].scrollIntoView(true);", comment_overview)
        sleep(1)  # Đợi một chút để phần tử scroll vào view

        filter_button_found = False
        for xpath in xpaths:
            try:
                # Đợi cho đến khi nút bộ lọc sẵn sàng
                filter_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, xpath)))
                driver.execute_script("arguments[0].click();", filter_button)  # Sử dụng JS để click tránh ElementClickInterceptedException
                print("Đã click vào bộ lọc.")
                filter_button_found = True
                sleep(5)  # Đợi sau khi click vào bộ lọc
                break  # Thoát khỏi vòng lặp nếu thành công
            except:
                # Nếu không tìm thấy nút bằng XPATH này, tiếp tục thử XPATH tiếp theo
                continue

        if not filter_button_found:
            print("Không thể tìm thấy hoặc click vào bộ lọc bình luận.")
            return comments_list

    except Exception as e:
        print(f"Không thể tìm thấy hoặc click vào bộ lọc bình luận. Lỗi: {e}")
        return comments_list
    
    while True:        
        try:
            comments = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.shopee-product-rating')))
            for comment_element in comments:
                # Sử dụng find_elements và kiểm tra kích thước danh sách để tránh chờ đợi không cần thiết
                customer_name_elements = comment_element.find_elements(By.CLASS_NAME, 'shopee-product-rating__author-name')
                customer_name = customer_name_elements[0].text.strip() if customer_name_elements else None
                print("customer_name:", customer_name)

                time_comment_elements = comment_element.find_elements(By.CLASS_NAME, 'shopee-product-rating__time')
                time_comment = time_comment_elements[0].text.strip() if time_comment_elements else None
                print("time_comment:", time_comment)

                content_elements = comment_element.find_elements(By.CSS_SELECTOR, "div[style='position: relative; box-sizing: border-box; margin: 15px 0px; font-size: 14px; line-height: 20px; color: rgba(0, 0, 0, 0.87); word-break: break-word; white-space: pre-wrap;']")
                content = content_elements[0].text if content_elements else None
                print("content:", content)

                thank_count_elements = comment_element.find_elements(By.CSS_SELECTOR, '.shopee-product-rating__like-count')
                thank_count = thank_count_elements[0].text if thank_count_elements else None
                print("thank_count:", thank_count)

                svg_elements = comment_element.find_elements(By.CSS_SELECTOR, '.shopee-product-rating__rating svg')
                stars = 5 - sum(1 for svg in svg_elements if 'none' in svg.get_attribute('style')) if svg_elements else None
                print("stars:", stars)

                comment_info = {
                    'Platform': 'Shopee',
                    'URL': link,
                    'customer_name': customer_name,
                    'time_comment': time_comment,
                    'content': content,
                    'thank_count': thank_count,
                    'rating': stars
                }
                comments_list.append(comment_info)

            if not end_page(driver):
                print('Finished 1 page.')
                try:
                    next_page = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main > div > div:nth-child(3) > div:nth-child(1) > div > div > div > div.wAMdpk > div > div.page-product__content--left > div:nth-child(2) > div > div > div.product-ratings__list > div.shopee-page-controller.product-ratings__page-controller > button.shopee-icon-button.shopee-icon-button--right')))
                    next_page.click()
                    sleep(5)
                except TimeoutException:
                    print("Reached the end or failed to find the next button.")
                    break
            else:
                print('Last page.')
                break
        except TimeoutException:
            print("Không tìm thấy bình luận cho link này. Chuyển sang link tiếp theo.")
            break
    return comments_list


def end_page(driver):
    try:
        # Correctly find the container with both classes
        buttons_container = driver.find_element(By.CSS_SELECTOR, 'div.shopee-page-controller.product-ratings__page-controller')
        
        # Get all the buttons within the container
        buttons = buttons_container.find_elements(By.TAG_NAME, 'button')
        
        # Find the "Next" button by its class name
        next_button = driver.find_element(By.CSS_SELECTOR, "button.shopee-icon-button.shopee-icon-button--right")
        
        # Check if the "Prev" button exists and analyze its class to determine the state
        if next_button and len(buttons) > 1:  # Ensure there is more than one button (Next and possibly Prev)
            prev_button = buttons[-2]  # Assuming the second-last button is the "Prev" button
            prev_button_class = prev_button.get_attribute('class')
            
            # Determine if the "Prev" button indicates a non-clickable state or a primary action
            if "shopee-button-no-outline--non-click" in prev_button_class:
                # If "Prev" is non-clickable, it means we're on the first page, so more pages are likely
                return False
            elif "shopee-button-solid--primary" in prev_button_class:
                # If "Prev" is primary, and since we're checking "Next" presence, determine by its state
                return not "shopee-button-disabled" in next_button.get_attribute('class')
        else:
            # If there's only one button or "Next" button state implies no more pages
            return True
    except NoSuchElementException:
        # If any elements required for determination are missing, assume end of pagination
        return True

In [7]:
df = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang TMĐT\Shopee\product_smartphone_shopee_19-3.csv")
linkss = df['URL']
linkss.to_list()


['https://shopee.vn/Apple-iPhone-15-Plus-128GB-i.288286284.23555130713?sp_atk=d60a40a1-53ca-40c1-a3f3-38bbc882deee&xptdk=d60a40a1-53ca-40c1-a3f3-38bbc882deee',
 'https://shopee.vn/Apple-iPhone-15-Pro-Max-256GB-Ch%C3%ADnh-h%C3%A3ng-VN-A-i.308461157.23354704062?sp_atk=98f5b9c0-3fb9-4706-b764-4778583d8f2f&xptdk=98f5b9c0-3fb9-4706-b764-4778583d8f2f',
 'https://shopee.vn/Apple-iPhone-15-Pro-128GB-Ch%C3%ADnh-h%C3%A3ng-VN-A-i.308461157.22180197277?sp_atk=256de47f-d920-445b-b76a-700a3ede13d5&xptdk=256de47f-d920-445b-b76a-700a3ede13d5',
 'https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Apple-iPhone-15-Pro-Max-256GB-i.288286284.21993222972?sp_atk=17f3217a-2001-42fa-a6d6-18573aaa7ac9&xptdk=17f3217a-2001-42fa-a6d6-18573aaa7ac9',
 'https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Apple-iPhone-15-Pro-128GB-i.88201679.21184086567?sp_atk=381c5d47-ba0d-41d3-9b03-24ac7a6bb240&xptdk=381c5d47-ba0d-41d3-9b03-24ac7a6bb240',
 'https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Apple-iPhone-15-Pro-Max-256GB

In [11]:
# Khởi tạo undetected_chromedriver với các tuỳ chỉnh
options = uc.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
driver.implicitly_wait(20)

# Thêm cookies vào trình duyệt
add_cookies(driver, cookies)

comments = []
for link in linkss:
    comments.extend(get_comment(driver, link))

# Sau khi đã lấy xong dữ liệu, đóng trình duyệt
driver.quit()

# Tạo DataFrame từ danh sách comments và xuất ra CSV
comments_df = pd.DataFrame(comments)
comments_df.to_csv('comments_shopee_19-3.csv', mode='a', header=not os.path.exists('comments_shopee_19-3.csv'), index=False, encoding='utf-8-sig')
print(comments_df)

Cookies added successfully.
https://shopee.vn/Apple-iPhone-15-Plus-128GB-i.288286284.23555130713?sp_atk=d60a40a1-53ca-40c1-a3f3-38bbc882deee&xptdk=d60a40a1-53ca-40c1-a3f3-38bbc882deee
Đã click vào bộ lọc.
customer_name: anhanh3868
time_comment: 2023-12-18 09:41 | Phân loại hàng: Pink
content: Đúng với mô tả: đúng
Tính năng nổi bật: không có
Chất lượng sản phẩm: good
Shop giao hàng nguyên đai nguyên kiện có dán băng dính của shop, người giao hàng xác nhận đúng chính chủ mới được nhận hàng
thank_count: 9
stars: 5
customer_name: g*****4
time_comment: 2024-01-17 14:14 | Phân loại hàng: Pink
content: Chất lượng sản phẩm: qua tuyet voi
Đúng với mô tả: chuan mo ta
Tính năng nổi bật: đầy đủ tính năng
Giao hàng nhanh đóng cẩn thận
thank_count: 7
stars: 5
customer_name: hadang443
time_comment: 2024-01-30 17:23 | Phân loại hàng: Green
content: Đã nhận hàng, đóng gói rất cẩn thận. Ip thì rất uy tín rồi khỏi bàn về chất lượng. Mua đúng ngày sale nên rẻ đc 2 triệu, phấn khởi. Sau mua mình cũng sẽ mu

In [13]:
df1 = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang TMĐT\Shopee\product_smartphone_shopee_19-3.csv")
df2 = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang TMĐT\Shopee\comments_shopee_19-3.csv")

# Merging the two datasets on 'id' from data_smartphone and 'product_id' from comments
merged_data = pd.merge(df1, df2, on='URL', how='inner')
merged_data.to_csv('Shopee_Smartphone_19-3.csv', index=False, encoding='utf-8-sig')
# Display the first few rows of the merged dataset for verification
merged_data.head()

,Platform_x,URL,Date Crawled,Store,Product Title,brand,Current Price,Discounted Price,Discount,Available Colors,sold_quanlity,rating_averrage,rating_count,Platform_y,customer_name,time_comment,content,thank_count,rating
0,Shopee,https://shopee.vn/Apple-iPhone-15-Plus-128GB-i...,3/19/2024,ShopDunk Official Store,Apple iPhone 15 Plus 128GB,Apple,₫29.990.000,₫23.590.000,21% GIẢM,"['Blue', 'Black', 'Pink', 'Green', 'Yellow']",354,4.9,77,Shopee,anhanh3868,2023-12-18 09:41 | Phân loại hàng: Pink,Đúng với mô tả: đúng\nTính năng nổi bật: không...,9,5
1,Shopee,https://shopee.vn/Apple-iPhone-15-Plus-128GB-i...,3/19/2024,ShopDunk Official Store,Apple iPhone 15 Plus 128GB,Apple,₫29.990.000,₫23.590.000,21% GIẢM,"['Blue', 'Black', 'Pink', 'Green', 'Yellow']",354,4.9,77,Shopee,g*****4,2024-01-17 14:14 | Phân loại hàng: Pink,Chất lượng sản phẩm: qua tuyet voi\nĐúng với m...,7,5
2,Shopee,https://shopee.vn/Apple-iPhone-15-Plus-128GB-i...,3/19/2024,ShopDunk Official Store,Apple iPhone 15 Plus 128GB,Apple,₫29.990.000,₫23.590.000,21% GIẢM,"['Blue', 'Black', 'Pink', 'Green', 'Yellow']",354,4.9,77,Shopee,hadang443,2024-01-30 17:23 | Phân loại hàng: Green,"Đã nhận hàng, đóng gói rất cẩn thận. Ip thì rấ...",Hữu Ích?,5
3,Shopee,https://shopee.vn/Apple-iPhone-15-Plus-128GB-i...,3/19/2024,ShopDunk Official Store,Apple iPhone 15 Plus 128GB,Apple,₫29.990.000,₫23.590.000,21% GIẢM,"['Blue', 'Black', 'Pink', 'Green', 'Yellow']",354,4.9,77,Shopee,namu61993,2023-10-22 08:03 | Phân loại hàng: Pink,Thật là tuyệt vời khi nhận được em nó. Màu đẹp...,13,5
4,Shopee,https://shopee.vn/Apple-iPhone-15-Plus-128GB-i...,3/19/2024,ShopDunk Official Store,Apple iPhone 15 Plus 128GB,Apple,₫29.990.000,₫23.590.000,21% GIẢM,"['Blue', 'Black', 'Pink', 'Green', 'Yellow']",354,4.9,77,Shopee,johnnytrong92,2024-03-09 11:14 | Phân loại hàng: Green,Rất hài lòng về chất lượng máy và cách đóng gó...,1,5
